Introduction
-----------------
This notebooks shows some features of the mm3dtestdata tools.

Usage
------

The mm3dtestdata is a set of routines that allow one to build a 3D dataset consisting of
- background (class 0)
- matrix (class 1)
- spheres (class 2)
- ellipsoids (class 3)

The spheres and ellipoids - balls and eggs -  are placed at random in a cube, and are embedded in a matrix.
The associated class maps can be used to compute multidimensional maps, for instance simulating workflows seen in X-ray tomography, X-ray fluoresence tomography or other spectral imaging approaches.

Overcoming challenges in data fusion, due to different modalities and spatial mismatches, algorithms are needed to overcome these issues. These routines allow one to build test data one which one can test drive new approaches.

Below, basic usage is demonstrated.

Note
-------
This code is not build to be ultra fast, and for some calculations one might spend a bit of time.
At present, thius is accpetable: the idea is to generate test data and have available ground truth data against which to validate ones algorithms.

In [1]:
import numpy as np
import mm3dtestdata as mm3d
import matplotlib.pyplot as plt

Model Parameters
-------------

Some parameters of note :

- scale: the size of the cubic grid
- border: only place objects further away from the border
- radius: the radius of the sphere and corresponding equal-volume ellipsoid

Some parameters that govern derived data

- shake_rmsd: shake the objects to get a slightly different object between modalities
- sigma_spectral: blurring factor for our 'spectral' data
- sigma_xct: blurring factor for our tomography data

In [2]:
scale =128
border = 20
radius = 10

shake_rmsd = 2.0
sigma_sem = 1.0
sigma_xct = 3.0

Build the objects

In [3]:
obj = mm3d.balls_and_eggs(scale=scale, border=border, radius=radius, )
_, instance_map_0, class_map_0 = obj.fill()

In [ ]:
obj.perturb(shake=shake_rmsd, cut=None, erase=None)
_, instance_map_1, class_map_1 = obj.fill()
obj.reset()

Build class maps. Class labels are turned into one-hot-vectors, and are Gaussian smeared.

In [ ]:
class_map_semedx = mm3d.blur_it(class_map_0, sigma=sigma_sem)
class_map_xct = mm3d.blur_it(class_map_1, sigma=sigma_xct)

Show me

In [ ]:

# Create a figure
plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.imshow(class_map_semedx[1,scale//2,...])#, vmin=0, vmax = 1.0)
plt.colorbar(shrink=0.70)
plt.axis('off')  # Turn off axis numbers and labels
plt.title("class 1, slice 64 - spectral")

plt.subplot(1, 3, 2)
plt.imshow(class_map_semedx[2,scale//2,...])#, vmin=0, vmax = 1.0 )
plt.colorbar(shrink=0.70)
plt.axis('off')
plt.title("class 2, slice 64 - spectral")


plt.subplot(1, 3, 3)
plt.imshow(class_map_semedx[3,scale//2,...])#, vmin=0, vmax = 1.0)
plt.colorbar(shrink=0.70)
plt.axis('off')
plt.title("class 3, slice 64 - spectral")
plt.show()



# Create a figure
plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.imshow(class_map_xct[1,scale//2,...])#, vmin=0, vmax = 1.0)
plt.colorbar(shrink=0.70)
plt.axis('off')  # Turn off axis numbers and labels
plt.title("class 1, slice 64 - xct")

plt.subplot(1, 3, 2)
plt.imshow(class_map_xct[2,scale//2,...])#, vmin=0, vmax = 1.0 )
plt.colorbar(shrink=0.70)
plt.axis('off')
plt.title("class 2, slice 64 - xct")


plt.subplot(1, 3, 3)
plt.imshow(class_map_xct[3,scale//2,...])#, vmin=0, vmax = 1.0)
plt.colorbar(shrink=0.70)
plt.axis('off')
plt.title("class 3, slice 64 - xct")
plt.show()


Define spectra and densities for each class and compute the weighted maps

In [ ]:
tomo_map, _, _ = mm3d.build_material_maps_XCT_SEM_EDX(
    class_map_xct, "VEQI", ['Si', 'Ca', 'Fe', 'Al', ' O', ' K'])

_, sem_map, _ = mm3d.build_material_maps_XCT_SEM_EDX(
    class_map_semedx, "VEQI", ['Si', 'Ca', 'Fe', 'Al', ' O', ' K'])

We add noise as well to make life more interesting. Added noise consists of

$
(U_1^2+V_1^2)*\mathrm{factor*density} + (U_2^2+V_2^2)*\mathrm{dark_{factor}}
$

where U and V are Gaussian random variables wuith mean 0 and sigma 1.


In [ ]:
sem_map = sem_map + mm3d.noise(sem_map, 0.05, 0.0)
tomo_map = tomo_map + mm3d.noise(tomo_map, 0.01, 0.0)

Show me

In [ ]:

# Create a figure
plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.imshow(sem_map[0,scale//2,...])#, vmin=0, vmax = 0.5)
plt.colorbar(shrink=0.70)
plt.axis('off')  # Turn off axis numbers and labels

plt.subplot(1, 3, 2)
plt.imshow(sem_map[2,scale//2,...])#, vmin=0, vmax = 0.05)
plt.colorbar(shrink=0.70)
plt.axis('off')


plt.subplot(1, 3, 3)
plt.imshow(sem_map[4,scale//2,...])#, vmin=0, vmax = 0.10)
plt.colorbar(shrink=0.70)
plt.axis('off')

# Display the plot
plt.show()

# Create a figure
plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.imshow(tomo_map[0,scale//2,...])#, vmin=0.05, vmax = 0.55)
plt.colorbar(shrink=0.70)
plt.axis('off')  # Turn off axis numbers and labels

plt.subplot(1, 3, 2)
plt.imshow(tomo_map[0,scale//2,...])#, vmin=0.05, vmax = 0.55)
plt.colorbar(shrink=0.70)
plt.axis('off')


plt.subplot(1, 3, 3)
plt.imshow(tomo_map[0,scale//2,...])#, vmin=0.05, vmax = 0.55)
plt.colorbar(shrink=0.70)
plt.axis('off')

# Display the plot
plt.show()